<a href="https://colab.research.google.com/github/olaviinha/SloppyNoto/blob/master/crawlers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font face="Trebuchet MS" size="6">Data Crawlers &nbsp;&nbsp;<a href="https://github.com/olaviinha/SloppyNoto" target="_blank"><font color="#999" size="4">Github</font></a><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font size="3" color="#999"><a href="https://inha.se" target="_blank"><font color="#999">O. Inha</font></a></font></font>


This utility is to accompany [Sloppy Noto](https://github.com/olaviinha/SloppyNoto) to provide a bit of aid in the quest of hunting down possible data file candidates for data to audio conversion.

<font color="#9d9">**HTTP Crawler**</font> crawls ftp-like websites (_"Index of /something"_). Due to federal policies, some instances such as ESA and NASA have changed some of their data archive servers from FTP to HTTP(S). This crawler will take care of that. HTTP Crawler crawls 3 levels of subdirectories in depth. **Note**: This has only been tested with HTTP indexes of some ESA and NASA servers. While HTTP indexes may _look_ quite similar, their HTML structure can be very different, making this crawler subject to failuress. Feel free to raise issue in Github in such occurrences.

<font color="#99d">**FTP Crawler**</font> crawls FTP directories as anonymous. FTP Crawler crawls **all** levels of subdirectories in depth, and it is **not recommended** to crawl top level, or any upper level directories (such as a directory containing _all data of an entire space mission_), for such crawling may take literally **hours** to complete.

<font color="#9dd">**ZIP Crawler**</font> crawls individual compressed and uncompressed archive files (.zip, .tar, .tar.gz). ZIP Crawler crawls all levels of subdirectories within the zip. To use data files from inside crawled zips, you must copy the data files to your Google Drive using the enclosed <font color="#9dd">Drive Stasher</font> cell.

**All of the crawlers** output:
- **Recursive** list of all files over 20 MB in size from whatever you're crawling.
- Categorized and sorted by file size.
- File types of your choosing highlighted with a green arrow (<font color="#9d9">`=>`</font>)

**Howto**
- Run _Setup & Settings_ cell after settings by clicking the play button on the left side of the cell.
- Input HTTP(S), FTP or ZIP address to the corresponding crawler cell and run.
- Copy/paste paths from the results to Sloppy Noto's `data_file` field.
- To use data files found from inside crawled ZIP files, use <font color="#9dd">Drive Stasher</font> cell to copy those files from ZIPs to your Google Drive. Then use the provided Drive path in Sloppy Noto.

**Note**
- Everything this utility does is **guessing** based on file names and sizes, thus all results are fully **indicative**. It is certainly not set in stone that any files that these crawlers find are going to work in Sloppy Noto.

# Setup & Settings


In [ ]:
#@markdown <small>Highlight these extensions in all crawl results. Case-insensitive, comma-separated list of file extensions, including period.</small>
highlight_extensions = ".csv, .tsv, .tab, .lst, .log, .asc" #@param {type:"string"}



#@markdown <small>Path to a directory in your Google Drive. Everything you choose to save, will be saved in this directory. Relative to your Drive root.</small>
save_dir = "" #@param {type:"string"}

#@markdown <small>Save resulting file lists from your crawls as well as source information of copied files as .txt files (to the directory set above). May come in handy in the future, not having to crawl the same things time after time.</small>
save_output_txt = False #@param {type:"boolean"}

save_txt = save_output_txt


from google.colab import output
import os

force_setup = False

  
# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  pip_packages = 'ftputil'
  %cd /content/
  !pip -q install import-ipynb {pip_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
# Mount Drive
if not os.path.isdir('/content/drive') and force_setup == False:
  from google.colab import drive
  drive.mount('/content/drive')
# Drive symlink
if not os.path.isdir('/content/mydrive') and force_setup == False:
  os.symlink('/content/drive/My Drive', '/content/mydrive')
  drive_root_set = True

import import_ipynb
from inhagcutils import *
drive_root = '/content/mydrive/'
dir_tmp = '/content/tmp/'
if not os.path.isdir(dir_tmp):
  create_dirs([dir_tmp])

import sys
import ftplib
import ftputil
import fnmatch


#size_limits = [1000, 500, 200, 100, 50, 20]
size_limits = [1000, 500, 200, 100, 50, 20]
b = 1000000
size_limits = [limit*b for limit in size_limits]

def fix_extensions(input_extensions):
  extensions = input_extensions.split(',')
  gz_extensions = [ext+'.gz' for ext in extensions]
  extensions.extend(gz_extensions)
  extensions = [ext.lower() for ext in extensions]
  extensions.extend([ext.upper() for ext in extensions])
  return tuple(extensions)

def apnd(content):
  global log_all, txt
  log_all = open(txt, 'a+')
  log_all.write(content)
  log_all.close()

def print_filelist(list, title, ftp=True):
  global log_all, save_txt
  total = len(list)
  op(c.title, '\n'+title+':\n')
  if save_txt == True:
    apnd('\n'+title+'\n')
  list.sort(key=lambda tup: tup[0], reverse=True)
  i = 0
  for s, f in list:
    if ftp == True:
      line = str('{:.2f}'.format(round(s/b, 2)))+' MB: ftp://'+ftp_host+f
    else:
      line = str('{:.2f}'.format(round(s/b, 2)))+' MB: '+f
    s_line = line
    if f.endswith(extensions):
      if i < 500:
        op(c.ok, '=>', line)
      s_line = '=> '+line
    else:
      if i < 500:
        print(line)
    if i == 500:
      remaining = total-500
      print('...and', str(remaining), 'more files of this size scale were found.')
      op(c.warn, '\nPrinting was stopped at 500.', 'Full file list is saved in a .txt file if you have save_txt checked.')
    if save_txt == True:
      apnd(s_line+'\n')
    i += 1
      
def retrieve_print_filelist(basedir, ftp=True):
  global size_limits, b
  all_files = []
  files_by_size = [[] for _ in size_limits]

  if ftp == True:
    recursive = host.walk(basedir, topdown=True, onerror=None) # recursive search 
  else:
    recursive = os.walk(basedir, topdown=True)
  print('Recursive file list retrieval in progress...')
  files_found = 0
  for root, dirs, files in recursive:
    for name in files:
      fullpath = os.path.join(root, name)
      if ftp == True:
        size = host.path.getsize(fullpath)
      else:
        size = os.path.getsize(fullpath)
      all_files.append([size, fullpath])

      for i, limit in enumerate(size_limits):
        if i == 0:
          if size > limit:
            files_by_size[i].append([size, fullpath])
            files_found += 1
        else:
          if size > limit and size < size_limits[i-1]:
            files_by_size[i].append([size, fullpath])
            files_found += 1
  output.clear()
  op(c.ok, '\nResults\n\n')

  if files_found == 0:
    op(c.fail, 'No suitable files found.')
  else:
    for i, filelist in enumerate(files_by_size):
      if len(filelist) > 0:
        if i == 0:
          title = 'Over 1 GB'
        else:
          title = str(round(size_limits[i]/b))+'-'+str(round(size_limits[i-1]/b))+' MB'
        print_filelist(filelist, title, ftp=ftp)
    if save_txt == True:
      log_all.close()

extensions = fix_extensions(highlight_extensions)
save_path = fix_path(drive_root+save_dir)

output.clear()
op(c.ok, 'Setup finished.')


#<font color="#9d9">HTTP Crawler</font>

In [37]:
#@markdown HTTP Crawler is meant strictly for http(s) addresses with an FTP-like view (normally titled _"Index of /something"_).
#@markdown Crawls **3 levels** deep in the file system tree structure.
web_url = "https://heasarc.gsfc.nasa.gov/FTP/" #@param {type:"string"}

#import pprint from pprint
import requests
import urllib
from bs4 import BeautifulSoup

if save_txt == True:
  web_host = slug(web_url.replace('http://', '').replace('https://', '').split('/')[0])
  webdir = web_url.replace(webhost, '').replace('/', '_')
  txt = save_path+'filelist_'+webhost+'_'+path_leaf(web_url)+'_'+rnd_str(4)+'.txt'
  apnd('WEB URL: '+web_url+'\n')
  apnd('WEB host: '+web_host+'\n')

def get_url_paths(url, ext='', params={}):
  response = requests.get(url, params=params)
  if response.ok:
    response_text = response.text
  else:
    return response.raise_for_status()
  soup = BeautifulSoup(response_text, 'html.parser')
  
  #link = [url + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]
  #parent = [url + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]
  li = []
  for link in soup.find_all('a'):
    href = link.get('href')
    if href != '.' and href != '..' and not href.startswith('?') and not href.startswith('/') and href != 'FTP/':
      fullhref = url + href
      meta = link.next_sibling.split()
      if len(meta) > 0:
        size = meta[2]
      else:
        size = 0
      li.append([size, fullhref])
  return li
  #return parent

def crawl_dirs(links):
  dir_contents = []
  ndx = 0
  for i, link in enumerate(links):
    href = link[1]
    if href.endswith('/'):
      if not href.endswith('../') and not href.endswith('./'):
        dir_contents.append( get_url_paths(href) )
        ndx += 1
  return dir_contents

print('Recursive file list retrieval in progress...')
links = get_url_paths(web_url, '')
subs = crawl_dirs(links)

subsubs = []
for i, sub in enumerate(subs):
  subsubs.append(crawl_dirs(sub))

def size_filter(links):
  global size_limits, b
  files_by_size = [[] for _ in size_limits]
  files_found = 0
  for link in links:
    size = link[0]
    if size != '-':
      shorts = ('M', 'K', 'G')
      if size.endswith(shorts):
        if '.' in size:
          size = size.replace('.', '').replace('K', '00').replace('M', '00000').replace('G', '00000000')
        else:
          size = size.replace('K', '000').replace('M', '000000').replace('G', '000000000')
      size = int(size)
    href = link[1]
    if not href.endswith('/'):
      fullpath = href
      for i, limit in enumerate(size_limits):
        if i == 0:
          if size > limit:
            files_by_size[i].append([size, fullpath])
            files_found += 1
        else:
          if size > limit and size < size_limits[i-1]:
            files_by_size[i].append([size, fullpath])
            files_found += 1

  output.clear()
  op(c.ok, '\nResults\n\n')
  if files_found == 0:
    op(c.fail, 'No suitable files found.')
  else:
    for i, filelist in enumerate(files_by_size):
      if len(filelist) > 0:
        if i == 0:
          title = 'Over 1 GB'
        else:
          title = str(round(size_limits[i]/b))+'-'+str(round(size_limits[i-1]/b))+' MB'
        print_filelist(filelist, title, ftp=False)

all_fucking_links = []

#lvl 1
if len(links) > 0:
  all_fucking_links.extend(links)

#lvl 2
if len(subs) > 0:
  for sub in subs:
    all_fucking_links.extend(sub)

#lvl 3
if len(subsubs) > 0:
  for subsub in subsubs:
    for subsubst in subsub:
      all_fucking_links.extend(subsubst)

size_filter(all_fucking_links)




Results



Over 1 GB:

30000.00 MB: https://heasarc.gsfc.nasa.gov/FTP/mod_tar/AwfO40
29000.00 MB: https://heasarc.gsfc.nasa.gov/FTP/mod_tar/VeYsyP
26000.00 MB: https://heasarc.gsfc.nasa.gov/FTP/mod_tar/HXArqH
8000.00 MB: https://heasarc.gsfc.nasa.gov/FTP/integral/caldb/ic.tar.gz
8000.00 MB: https://heasarc.gsfc.nasa.gov/FTP/integral/caldb/ic.tar.gz.old
2000.00 MB: https://heasarc.gsfc.nasa.gov/FTP/xarm/simulation/xarm_prop_16Aug2017_V001_src.tgz
1800.00 MB: https://heasarc.gsfc.nasa.gov/FTP/retrieve/w3browse/w3browse-144601.tar
1800.00 MB: https://heasarc.gsfc.nasa.gov/FTP/retrieve/w3browse/w3browse-145012.tar
1600.00 MB: https://heasarc.gsfc.nasa.gov/FTP/software/dts/dasdbtest.tar.gz
1500.00 MB: https://heasarc.gsfc.nasa.gov/FTP/retrieve/w3browse/w3browse-176742.tar
1500.00 MB: https://heasarc.gsfc.nasa.gov/FTP/retrieve/w3browse/w3browse-177451.tar
1500.00 MB: https://heasarc.gsfc.nasa.gov/FTP/retrieve/w3browse/w3browse-216612.tar
1500.00 MB: https://heasarc.gsfc.nasa.gov/FTP/retriev

#<font color="#99d">FTP Crawler</font>

In [ ]:
#@markdown FTP Crawler should not be used on upper level directories, or it may take **hours** to complete the listing. 
#@markdown Crawls to infinite depths in the file system tree structure.

ftp_url = "" #@param {type:"string"}

ftp_address = ftp_url.replace('ftp://', '')
ftp_host = ftp_address.split('/')[0]
basedir = ftp_address.replace(ftp_host, '')

if save_txt == True:
  txt = save_path+'filelist_'+ftp_host+'_'+rnd_str(4)+'.txt'
  apnd('FTP URL: '+ftp_url+'\n')
  apnd('FTP host: '+ftp_host+'\n')
  apnd('FTP dir: '+basedir+'\n\n')
output.clear()
op(c.title, 'Logging in to '+ftp_host+'...')
host = ftputil.FTPHost(ftp_host, 'anonymous', 'anonymous@domain.com')

retrieve_print_filelist(basedir, ftp=True)

#<font color="#9dd">ZIP Crawler</font>

In [38]:
#@markdown ZIP Crawler is to be used for compressed files (`.zip`, `.gz`, `.tar.gz`). Crawls to infinite depths within the zip's enclosing tree structure.
zip_url = "https://heasarc.gsfc.nasa.gov/FTP/retrieve/w3browse/w3browse-114752.tar" #@param {type:"string"}
import shutil

if save_txt == True:
  zipfile = basename(zip_url)
  txt = save_path+'filelist_zip_'+zipfile+'_'+rnd_str(4)+'.txt'
  apnd('ZIP URL: '+zip_url+'\n\n')

def is_zipzip(path):
  return path_ext(path).lower() == '.zip'
  
def is_gz(path):
  return path_ext(path).lower() == '.gz'

zip_ext = path_ext(zip_url, True)
#id = rnd_str(6)
wfile = slug(basename(zip_url))
wdir = dir_tmp+wfile+'/'
wext = path_ext(zip_url)
file_path = wdir+wfile+wext
if not os.path.isdir(wdir):
  !mkdir {wdir}
  !wget {zip_url} -O {file_path}
  shutil.unpack_archive(file_path, path_dir(file_path))
  !rm {file_path}
retrieve_print_filelist(wdir, ftp=False)



Results



20-50 MB:

38.60 MB: /content/tmp/w3browse-114752/00013636009/uvot/products/sw00013636009u_sk.img.gz
35.97 MB: /content/tmp/w3browse-114752/00013636010/uvot/products/sw00013636010u_sk.img.gz
33.33 MB: /content/tmp/w3browse-114752/00013636008/uvot/products/sw00013636008u_sk.img.gz
32.31 MB: /content/tmp/w3browse-114752/00013636004/uvot/products/sw00013636004u_sk.img.gz
32.27 MB: /content/tmp/w3browse-114752/00013636006/uvot/products/sw00013636006u_sk.img.gz
24.20 MB: /content/tmp/w3browse-114752/00013636007/uvot/products/sw00013636007u_sk.img.gz
23.88 MB: /content/tmp/w3browse-114752/00013636002/uvot/products/sw00013636002u_sk.img.gz
23.01 MB: /content/tmp/w3browse-114752/00013636001/uvot/products/sw00013636001u_sk.img.gz
21.47 MB: /content/tmp/w3browse-114752/00013636003/uvot/products/sw00013636003u_sk.img.gz
21.39 MB: /content/tmp/w3browse-114752/00013636005/uvot/products/sw00013636005u_sk.img.gz


##<font color="#9dd">Drive Stasher</font>

In [ ]:
#@markdown Use this cell to stash data files from ZIP Crawler to your Drive.<br>
#@markdown <small>Paste file path and run cell.</small>
copy_file = "" #@param {type:"string"}

!cp {copy_file} {save_path}
op(c.ok, 'Copied to Drive:', path_leaf(copy_file)+'\n')
op(c.ok, 'You may use the following path as data_file in Sloppy Noto as soon as the file is synced in your Drive:\n', save_path+path_leaf(copy_file))
if save_txt == True:
  apnd('\nCopied '+path_leaf(copy_file)+' to '+save_path)